In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Pairwise_Final/pairwise')

import numpy as np
import time
from os import listdir
import cv2
from scipy.io import savemat
import matplotlib.pyplot as plt
from tabulate import tabulate
from compute_transform import compute_transform, compute_transform_north, compute_transform_west


srcs = [
    # '/content/drive/MyDrive/Pairwise_Final/pairwise/DATA/Tak_Dataset-Corrected1/026-01-91-Corrected',
    # '/content/drive/MyDrive/Pairwise_Final/pairwise/DATA/Tak_Dataset-Corrected1/051-04-80-Corrected',
    # '/content/drive/MyDrive/Pairwise_Final/pairwise/DATA/Tak_Dataset-Corrected1/156-01-86-Corrected',
    # '/content/drive/MyDrive/Pairwise_Final/pairwise/DATA/Tak_Dataset-Corrected1/194-01-70-Corrected',
    # '/content/drive/MyDrive/Pairwise_Final/pairwise/DATA/Tak_Dataset-Corrected1/234-01-67-Corrected',

    # '/content/drive/MyDrive/Pairwise_Final/pairwise/DATA/Tak_Dataset-Corrected2/31-01-Corrected',
    # '/content/drive/MyDrive/Pairwise_Final/pairwise/DATA/Tak_Dataset-Corrected2/33-03-Corrected',
    # '/content/drive/MyDrive/Pairwise_Final/pairwise/DATA/Tak_Dataset-Corrected2/36-01-Corrected',
    # '/content/drive/MyDrive/Pairwise_Final/pairwise/DATA/Tak_Dataset-Corrected2/49-01-Corrected',
    # '/content/drive/MyDrive/Pairwise_Final/pairwise/DATA/Tak_Dataset-Corrected2/53-03-Corrected'


    # '/content/drive/MyDrive/Pairwise_Final/pairwise/DATA/Shading/026-01-91'
    # '/content/drive/MyDrive/Pairwise_Final/pairwise/DATA/Shading/051-04-80',
    # '/content/drive/MyDrive/Pairwise_Final/pairwise/DATA/Shading/156-01-86',
    '/content/drive/MyDrive/Pairwise_Final/pairwise/DATA/Shading/194-01-70'
    # '/content/drive/MyDrive/Pairwise_Final/pairwise/DATA/Shading/234-01-67',

    # '/content/drive/MyDrive/Pairwise_Final/pairwise/DATA/Shading/31-01',
    # '/content/drive/MyDrive/Pairwise_Final/pairwise/DATA/Shading/53-03',
    # '/content/drive/MyDrive/Pairwise_Final/pairwise/DATA/Shading/49-01',
    # '/content/drive/MyDrive/Pairwise_Final/pairwise/DATA/Shading/33-03',
    # '/content/drive/MyDrive/Pairwise_Final/pairwise/DATA/Shading/36-01'


]
for s in srcs:
    main_loop(s)


194-01-70
  col:  0 / 9
  row:  0 / 8
  row:  1 / 8
  row:  2 / 8
  row:  3 / 8
  row:  4 / 8
  row:  5 / 8
  row:  6 / 8
  row:  7 / 8
  row:  8 / 8
  col:  1 / 9
  row:  0 / 8
  row:  1 / 8
  row:  2 / 8
  row:  3 / 8
  row:  4 / 8
  row:  5 / 8
  row:  6 / 8
  row:  7 / 8
  row:  8 / 8
  col:  2 / 9
  row:  0 / 8
  row:  1 / 8
  row:  2 / 8
  row:  3 / 8
  row:  4 / 8
  row:  5 / 8
  row:  6 / 8
  row:  7 / 8
  row:  8 / 8
  col:  3 / 9
  row:  0 / 8
  row:  1 / 8
  row:  2 / 8
  row:  3 / 8
  row:  4 / 8
  row:  5 / 8
  row:  6 / 8
  row:  7 / 8
  row:  8 / 8
  col:  4 / 9
  row:  0 / 8
  row:  1 / 8
  row:  2 / 8
  row:  3 / 8
  row:  4 / 8
  row:  5 / 8
  row:  6 / 8
  row:  7 / 8
  row:  8 / 8
  col:  5 / 9
  row:  0 / 8
  row:  1 / 8
  row:  2 / 8
  row:  3 / 8
  row:  4 / 8
  row:  5 / 8
  row:  6 / 8
  row:  7 / 8
  row:  8 / 8
  col:  6 / 9
  row:  0 / 8
  row:  1 / 8
  row:  2 / 8
  row:  3 / 8
  row:  4 / 8
  row:  5 / 8
  row:  6 / 8
  row:  7 / 8
  row:  8 / 8
  col:  7 

In [ ]:
def main_loop(source_directory):
    f_name = source_directory.split('/')[-1]
    print(f_name)
    files = listdir(source_directory)
    #natsort
    Start = 0
    hop = 1
    End = 94

    file = files[Start:End:hop]
    file.sort()
    file = file[Start:90]
    # print(file)

    # Tak dataset 10*10 / human colon datset 29*21/ stem cell colony level1-2 10*10   level3 23*24/  small-phase or flourecent 5*5
    nb_horz_tiles = 9    # number of horizontal tiles
    nb_vert_tiles = 10    # number of vertical tiles

    # estimated overlap
    # Tak dataset 25% / human colon datset 3% / stem cell colony level1-3 10% / small-phase or flourecent 20%
    OverlapY = 0.25
    OverlapX = 0.25

    index_matrix = np.arange(0,nb_vert_tiles*nb_horz_tiles)
    t = nb_vert_tiles
    nb_vert_tiles = nb_horz_tiles
    nb_horz_tiles = t

    #for Tak & stem cell colony dataset
    img_name_grid = np.reshape(file, [nb_vert_tiles, nb_horz_tiles])
    # print(tabulate(img_name_grid))
    index_matrix = np.reshape(index_matrix, [nb_vert_tiles, nb_horz_tiles])

    #for hunan colon dataset
    #img_name_grid = np.reshape(file, nb_vert_tiles, nb_horz_tiles).T
    #index_matrix = np.reshape(index_matrix, nb_vert_tiles, nb_horz_tiles)).T

    #img_name_grid = img_name_grid(sort(1:size(img_name_grid ,1),'descend'),:)
    #index_matrix = index_matrix(sort(1:size(index_matrix,1),'descend'),:)

    [nb_vert_tiles, nb_horz_tiles] = np.shape(img_name_grid)

    Ty_north = np.empty([nb_vert_tiles,nb_horz_tiles])
    Ty_north[:] = np.nan
    Tx_north = np.empty([nb_vert_tiles,nb_horz_tiles])
    Tx_north[:] = np.nan

    matchedNumb_north = np.empty([nb_vert_tiles,nb_horz_tiles])
    matchedNumb_north[:] = np.nan

    Ty_west = np.empty([nb_vert_tiles,nb_horz_tiles])
    Ty_west[:] = np.nan
    Tx_west = np.empty([nb_vert_tiles,nb_horz_tiles])
    Tx_west[:] = np.nan

    matchedNumb_west = np.empty([nb_vert_tiles,nb_horz_tiles])
    matchedNumb_west[:] = np.nan

    error_north = np.empty([nb_vert_tiles,nb_horz_tiles])
    error_north[:] = np.nan
    error_west = np.empty([nb_vert_tiles,nb_horz_tiles])
    error_west[:] = np.nan

    index_ImMatch_north = np.empty([nb_vert_tiles,nb_horz_tiles])
    index_ImMatch_north[:] = np.nan
    pointsPreviousNumb_north = np.empty([nb_vert_tiles,nb_horz_tiles])
    pointsPreviousNumb_north[:] = np.nan
    pointsNumb_north = np.empty([nb_vert_tiles,nb_horz_tiles])
    pointsNumb_north[:] = np.nan

    index_ImMatch_west = np.empty([nb_vert_tiles,nb_horz_tiles])
    index_ImMatch_west[:] = np.nan
    pointsPreviousNumb_west = np.empty([nb_vert_tiles,nb_horz_tiles])
    pointsPreviousNumb_west[:] = np.nan
    pointsNumb_west = np.empty([nb_vert_tiles,nb_horz_tiles])
    pointsNumb_west[:] = np.nan

    inliersNumb_north = np.empty([nb_vert_tiles,nb_horz_tiles])
    inliersNumb_north[:] = np.nan
    inliersNumb_west = np.empty([nb_vert_tiles,nb_horz_tiles])
    inliersNumb_west[:] = np.nan

    time_north  = np.empty([nb_vert_tiles,nb_horz_tiles])
    time_north[:] = np.nan
    time_west  = np.empty([nb_vert_tiles,nb_horz_tiles])
    time_west[:] = np.nan

    status_north  = np.empty([nb_vert_tiles,nb_horz_tiles])
    status_north[:] = np.nan
    status_west  = np.empty([nb_vert_tiles,nb_horz_tiles])
    status_west[:] = np.nan

    RMSE_north  = np.empty([nb_vert_tiles,nb_horz_tiles])
    RMSE_north[:] = np.nan
    RMSE_west  = np.empty([nb_vert_tiles,nb_horz_tiles])
    RMSE_west[:] = np.nan

    M_north  = np.empty([nb_vert_tiles,nb_horz_tiles,2,3])
    M_north[:] = np.nan
    M_west  = np.empty([nb_vert_tiles,nb_horz_tiles,2,3])
    M_west[:] = np.nan

    Kp = np.zeros([nb_vert_tiles,nb_horz_tiles,1000,2])
    des = np.zeros([nb_vert_tiles,nb_horz_tiles,1000,256])
    kp_stat = np.empty([nb_vert_tiles,nb_horz_tiles])
    kp_stat[:] = np.nan
    t1 = time.time()
    [M, N, channel] = np.shape(cv2.imread(source_directory+'/'+img_name_grid[0,0]))
    for j in range(nb_horz_tiles):
        print('  col: ', j,"/",nb_horz_tiles-1)
        for i in range(nb_vert_tiles):
            print('  row: ', i,"/",nb_vert_tiles-1)
            #read image from disk
            if channel == 3:
                I1 = cv2.cvtColor(cv2.imread(source_directory+'/'+img_name_grid[i,j]),cv2.COLOR_BGR2GRAY)
            else:
                I1 = cv2.imread(source_directory+'/'+img_name_grid[i,j])
            if i>0:
                # print("north")
                #compute translation north
                if channel == 3:
                    I2 = cv2.cvtColor(cv2.imread(source_directory+'/'+img_name_grid[i-1,j]),cv2.COLOR_BGR2GRAY)
                else:
                    I2 = cv2.imread(source_directory+'/'+img_name_grid[i-1,j])
                #compute_transform
                thr = (np.mean(I1.flatten()) + np.mean(I2.flatten()))/2
                # ret,th1 = cv2.threshold(I1,thr,255,cv2.THRESH_TOZERO_INV)
                # ret,th2 = cv2.threshold(I2,thr,255,cv2.THRESH_TOZERO_INV)
                th1 = I1
                th2 = I2
                [Ty_north[i,j], Tx_north[i,j], index_ImMatch_north[i,j], matchedNumb_north[i,j],
                    pointsPreviousNumb_north[i,j], pointsNumb_north[i,j], inliersNumb_north[i,j],
                    status_north[i,j], error_north[i,j],time_north[i,j]], kpI1, desI1, kpI2, desI2, M_north[i,j,:,:] = compute_transform_north(th1.copy(),th2.copy(),index_matrix[i,j], int(M*OverlapY),OverlapY,M,N)
                # [Ty_north[i,j], Tx_north[i,j], index_ImMatch_north[i,j], matchedNumb_north[i,j],
                #     pointsPreviousNumb_north[i,j], pointsNumb_north[i,j], inliersNumb_north[i,j],
                #     status_north[i,j], error_north[i,j],time_north[i,j]], kpI1, desI1, kpI2, desI2 = compute_transform(th1.copy(),th2.copy(),index_matrix[i,j])

            if j>0:
                # print("west")
                #compute translation west
                if channel == 3:
                    I2 = cv2.cvtColor(cv2.imread(source_directory+'/'+img_name_grid[i,j-1]),cv2.COLOR_BGR2GRAY)
                else:
                    I2 = cv2.imread(source_directory+'/'+img_name_grid[i,j-1])
                #compute_transform
                thr = (np.mean(I1.flatten()) + np.mean(I2.flatten()))/2
                # ret,th1 = cv2.threshold(I1,thr,255,cv2.THRESH_TOZERO_INV)
                # ret,th2 = cv2.threshold(I2,thr,255,cv2.THRESH_TOZERO_INV)
                th1 = I1
                th2 = I2
                [Ty_west[i,j], Tx_west[i,j], index_ImMatch_west[i,j], matchedNumb_west[i,j],
                    pointsPreviousNumb_west[i,j], pointsNumb_west[i,j], inliersNumb_west[i,j],
                    status_west[i,j], error_west[i,j],time_west[i,j]], kpI1, desI1, kpI2, desI2, M_west[i,j,:,:] = compute_transform_west(th1.copy(),th2.copy(),index_matrix[i,j], int(N*OverlapX),OverlapX,M,N)
                # [Ty_west[i,j], Tx_west[i,j], index_ImMatch_west[i,j], matchedNumb_west[i,j],
                #     pointsPreviousNumb_west[i,j], pointsNumb_west[i,j], inliersNumb_west[i,j],
                #     status_west[i,j], error_west[i,j],time_west[i,j]], kpI1, desI1, kpI2, desI2 = compute_transform(th1.copy(),th2.copy(),index_matrix[i,j])

    pairwise_time = time.time() - t1
    average_time = (np.mean(time_north[1:,:]) + np.mean(time_west[:,1:]))/2

    def extract_subregion(I, x, y):
        x = int(x)
        y = int(y)
        [h, w] = I.shape
        # print(x,y,h,w)
        if np.abs(x)>=w or np.abs(y)>=h:
            sub_I = np.array([])
            return sub_I
        x_st = 1 + x
        x_end = x_st + w - 1
        y_st = 1 + y
        y_end = y_st + h - 1
        # constrain to valid coords
        x_st = max(1, min(x_st, w));
        x_end = max(1, min(x_end, w));
        y_st = max(1, min(y_st, h));
        y_end = max(1, min(y_end, h));
        sub_I = I[y_st:y_end, x_st:x_end]
        # plt.figure()
        # plt.imshow(sub_I)
        return np.array(sub_I)


    for j in range(nb_horz_tiles):
        print('  col: ', j,"/",nb_horz_tiles)
        for i in range(nb_vert_tiles):
            print('  row: ', i,"/",nb_vert_tiles)
            #read image from disk
            if channel == 3:
                I1 = cv2.cvtColor(cv2.imread(source_directory+'/'+img_name_grid[i,j]),cv2.COLOR_BGR2GRAY)
            else:
                I1 = cv2.imread(source_directory+'/'+img_name_grid[i,j])
            if i>0:
                #compute RMSE  between overlaped region north

                if channel == 3:
                    I2 = cv2.cvtColor(cv2.imread(source_directory+'/'+img_name_grid[i-1,j]),cv2.COLOR_BGR2GRAY)
                else:
                    I2 = cv2.imread(source_directory+'/'+img_name_grid[i-1,j])
                I1 = cv2.normalize(I1, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                I2 = cv2.normalize(I2, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                if not np.isnan(Tx_north[i,j]) and not np.isnan(Ty_north[i,j]):
                  sub_I2 = extract_subregion(I2, Tx_north[i,j], Ty_north[i,j])
                  sub_I1 = extract_subregion(I1, -1*Tx_north[i,j], -1*Ty_north[i,j])
                  RMSE_north[i,j]  = np.sqrt(np.mean(np.power(sub_I2.flatten()-sub_I1.flatten(),2)))
                # print(np.sqrt(np.mean(np.power(sub_I2.flatten()-sub_I1.flatten(),2))))
            if j>0:
                #compute RMSE  between overlaped region  west
                if channel == 3:
                    I2 = cv2.cvtColor(cv2.imread(source_directory+'/'+img_name_grid[i,j-1]),cv2.COLOR_BGR2GRAY)
                else:
                    I2 = cv2.imread(source_directory+'/'+img_name_grid[i,j-1])
                I1 = cv2.normalize(I1, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                I2 = cv2.normalize(I2, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                if not np.isnan(Tx_west[i,j]) and not np.isnan(Ty_west[i,j]):
                  sub_I2 = extract_subregion(I2, Tx_west[i,j], Ty_west[i,j])
                  sub_I1 = extract_subregion(I1, -1*Tx_west[i,j], -1*Ty_west[i,j])
                  RMSE_west[i,j]  = np.sqrt(np.mean(np.power(sub_I2.flatten()-sub_I1.flatten(),2)))
                # print(np.sqrt(np.mean(np.power(sub_I2.flatten()-sub_I1.flatten(),2))))


    average_RMSE = (np.mean(RMSE_north[1:,:]) + np.mean(RMSE_west[:,1:]))/2

    mdic = {"tx_north":Tx_north,
            "tx_west" :Tx_west,
            "ty_north":Ty_north,
            "ty_west" :Ty_west,
            "tx_north":Tx_north,
            "matchedNumb_north": matchedNumb_north,
            "matchedNumb_west": matchedNumb_west,
            "error_north":error_north,
            "error_west":error_west,
            "index_ImMatch_north":index_ImMatch_north,
            "index_ImMatch_west":index_ImMatch_west,
            "pointsPreviousNumb_north":pointsPreviousNumb_north,
            "pointsPreviousNumb_west":pointsPreviousNumb_west,
            "pointsNumb_north":pointsNumb_north,
            "pointsNumb_west":pointsNumb_west,
            "inliersNumb_north":inliersNumb_north,
            "inliersNumb_west":inliersNumb_west,
            "time_north":time_north,
            "time_west":time_west,
            "status_north":status_north,
            "status_north":status_west,
            "RMSE_north":RMSE_north,
            "RMSE_west":RMSE_west,
            "pairwise_time":pairwise_time,
            "average_time":average_time,
            "average_RMSE":average_RMSE,
            }
    savemat(f_name+".mat", mdic)

In [ ]:
p0 = 2 + 3j
p00 = np.power(p0,2)
p01 = np.sqrt(p00)
print(p00,p01)

(-5+12j) (2+3j)
